In [0]:
import numpy as np
import random
import math
from sys import stdin, stdout, stderr
games= []
mc_moves = []

board_moves = []
Dx = 5
Dy = 5
SizePolicy = 3 * Dx * Dy

Empty = 0
White = 1
Black = 2

stringColor = ["Empty", "White", "Black"]

hashTable = []
for k in range (3):
    l = []
    for i in range (Dx):
        l1 = []
        for j in range (Dy):
            l1.append (int (random.random () * (2 ** 64)))
        l.append (l1)
    hashTable.append (l)

class Board(object):
    def __init__(self):
        self.turn = White
        self.h = 0
        self.board = np.zeros ((Dx, Dy))
        for i in range (0, 2):
            for j in range (0, Dy):
                self.board [i] [j] = White
        for i in range (Dx - 2, Dx):
            for j in range (0, Dy):
                self.board [i] [j] = Black

    def copy (self, b):
        self.turn = b.turn
        self.h = b.h
        for i in range (0, Dx):
            for j in range (0, Dy):
                self.board [i] [j] = b.board [i] [j]

    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                if self.board [i] [j] == self.turn:
                    for k in [-1, 0, 1]:
                        for l in [-1, 0, 1]:
                            m = Move (self.turn, i, j, i + k, j + l)
                            if m.valid (self):
                                moves.append (m)                                    
        return moves

    def print (self):
        for i in range (Dx):
            stdout.write ('|')
            for j in range (Dy):
                if self.board [i] [j] == White:
                    stdout.write ('w')
                if self.board [i] [j] == Black:
                    stdout.write ('b')
                if self.board [i] [j] == Empty:
                    stdout.write (' ')
                stdout.write ('|')
            stdout.write ('\n')
                
    def printMoves (self, moves):
        for i in range (0, len (moves)):
            print (moves [i])

    def play (self, move):
#        import pdb
#        pdb.set_trace()
        board_moves.append(self.board)
        col = int (self.board [move.x2] [move.y2])
        if col != Empty:
            self.h = self.h ^ hashTable [col] [move.x2] [move.y2]
        self.h = self.h ^ hashTable [move.color] [move.x2] [move.y2]

        games.append((move.x1, move.x2, move.y1, move.y2))
        self.h = self.h ^ hashTable [move.color] [move.x1] [move.y1]
        self.board [move.x2] [move.y2] = move.color
        self.board [move.x1] [move.y1] = Empty
        if (move.color == White):
            self.turn = Black
        else:
            self.turn = White

    def won (self, color):
        if (color == White):
            for i in range (0, Dy):
                if (self.board [Dx - 1] [i] == White):
#                    print('white wins')
                    return True
        else:
            for i in range (0, Dy):
                if (self.board [0] [i] == Black):
#                    print('black wins')
                    return True
        return False
        
    def playout (self):
        l = self.legalMoves ()
        while len (l) > 0 and not self.won (White) and not self.won (Black):
            indice = int (random.random () * len (l))
            m = l [indice]
            self.play (m)
            l = self.legalMoves ()
        if len (l) == 0:
            if self.turn == Black:
                return 1.0
            else:
                return 0.0
        if self.won (White):
            return 1.0
        else:
            return 0.0
        
    def flat (self, n):
        l = self.legalMoves ()
        best = l [0]
        bestScore = n + 1
        if self.turn == White:
            bestScore = -1.0
        b = Board ()
        for m in l:
            sum = 0
            for i in range (n):
                b.copy (self)
#                import pdb 
#                pdb.set_trace()
                b.play (m)
                r = b.playout ()
                sum = sum + r
            if (self.turn == White and sum > bestScore) or (self.turn == Black and sum < bestScore):
                    bestScore = sum
                    best = m
                    mc_moves.append(b)
        return best
    
    #getvalid: a binary vector of length 75, 1 for moves that are legal by Sofiene Omri
    def getvalid(self):
        l=self.legalMoves()
        K=75*[0]
        for m in l:
            i=m.code()
            K[i]=1
        res=np.array(K)
        return res

    #modified to get the given network output (policy and value) 
    def getOutputNetwork (self,NNeT):
        if self.turn == White:
            pi,v=NNeT.predict_netoutput(self)
            valids=self.getvalid()
            pi=pi*valids #keeps only valid moves and masks invalid ones
            sum_Pi = np.sum(pi)
            if sum_Pi > 0:
                pi /= sum_Pi    # renormalize to store policy probs
            pi=pi.flatten()
            pi=pi.tolist()
            return pi,v
        else:
            pi,v=NNeT.predict_netoutput(self)
            valids=self.getvalid()
            pi=pi*valids
            sum_Pi = np.sum(pi)
            if sum_Pi > 0:
                pi /= sum_Pi   
            pi=pi.flatten()
            pi=pi.tolist()
            return pi,v

    def descent (self, TT,NNeT):
        if self.h not in TT:
            output,v = self.getOutputNetwork (NNeT)
            l = []
            l.append (SizePolicy * [0])
            l.append (SizePolicy * [0.0])
            l.append (output)
            TT [self.h] = l
            #Return the estimated value of a state 
            return v
        elif self.won (White):
            return 1.0
        elif self.won (Black):
            return 0.0
        else:
            s = TT [self.h]
            l = self.legalMoves ()
            if len (l) == 0:
                if turn == White:
                    return 0.0
                return 1.0
            best = l [0]
            bestScore = -1000000000.0
            sumPlayouts = 0
            sumScores = 0.0
            for m in l:
                i = m.code ()
                sumPlayouts = sumPlayouts + s [0] [i]
                sumScores = sumScores + s [1] [i]
            mean = 0.0
            if sumPlayouts > 0:
                mean = sumScores / sumPlayouts
            for m in l:
                i = m.code ()
                mu = mean
                if s [0] [i] > 0:
                    mu = s [1] [i] / s [0] [i]
                if self.turn == Black:
                    mu = 1.0 - mu
                puct = mu + 0.3 * s [2] [i] * math.sqrt (sumPlayouts) / (1.0 + s [0] [i])
                if puct > bestScore:
                    bestScore = puct
                    best = m
            self.play (best)
            v = self.descent (TT,NNeT)
            #print('v self def',v)
            i = best.code ()
            s [0] [i] = s [0] [i] + 1
            s [1] [i] = s [1] [i] + v
            return v
        
    def PUCT (self, TT, NNeT, nbPlayouts = 200):
        b = Board ()
        for i in range (nbPlayouts):
            b.copy (self)
            b.descent (TT,NNeT)
        s = TT [self.h]
        # code modified to sample from improved network policy 
        action = np.random.choice(list(range(75)), p=s[2]) # sample action from improved policy
        l = self.legalMoves ()
        best=l[0]
        for m in l:
            i = m.code ()
            if action==i:
                best = m
        return best
    
class Move(object):
    def __init__(self, color, x1, y1, x2, y2):
        self.color = color
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
    
    def valid (self, board):
        if self.x2 >= Dx or self.y2 >= Dy or self.x2 < 0 or self.y2 < 0:
            return False
        if self.color == White:
            if self.x2 != self.x1 + 1:
                return False
            if board.board [self.x2] [self.y2] == Black:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1:
                    return True
                return False
            elif board.board [self.x2] [self.y2] == Empty:
                if (self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1 or self.y2 == self.y1):
                    return True
                return False
        elif self.color == Black:
            if self.x2 != self.x1 - 1:
                return False
            if board.board [self.x2] [self.y2] == White:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1:
                    return True
                return False
            elif board.board [self.x2] [self.y2] == Empty:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1 or self.y2 == self.y1:
                    return True
                return False
        return False
        
    def __str__(self):
        return stringColor [self.color] + '/' + str (self.x1) + '/' + str (self.y1) + '/' + str (self.x2) + '/' + str (self.y2)

    def code (self):
        direction = 1
        if self.y2 > self.y1:
            direction = 0
        if self.y2 < self.y1:
            direction = 2
        return 3 * (Dx * self.x1 + self.y1) + direction
            

In [0]:

#This function format a given board into a flattened list of 75 values
def Format_data(Input_Board): 
    W=np.where(Input_Board.board==2, 0, Input_Board.board)
    B=np.where(Input_Board.board==2, 1, Input_Board.board)
    E=np.where(Input_Board.board==0, 1, Input_Board.board)
    res=np.concatenate(((B-W), W, (E-Input_Board.board)), axis=0)
    rest=(res.tolist())
    rest = [item for sublist in rest for item in sublist]
    return rest

In [0]:

#Below is the developped neural network class for the breakthrough it contains a train, a predict and a Cnn_mve function 

from keras.models import *
from keras.layers import *
from keras.optimizers import *

class BreakthroughNNet():
    #Intialise the model
    def __init__(self):
        #############################params#######################################
        self.lr=0.001
        self.dropout=0.3
        self.num_filters=512
        self.board_x = 5
        self.board_y = 5
        self.action_size = 3 * self.board_x * self.board_y
        self.input_boards = Input(shape=(self.board_x, self.board_y,3)) 
        ############################network components###########################
        Layer_conv1 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(self.num_filters, 3, padding='same', use_bias=False)(self.input_boards)))         
        Layer_conv2 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(self.num_filters, 3, padding='same', use_bias=False)(Layer_conv1)))        
        Layer_conv3 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(self.num_filters, 3, padding='valid', use_bias=False)(Layer_conv2)))        
        Layer_conv4 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(self.num_filters, 3, padding='valid', use_bias=False)(Layer_conv3)))        
        Layer_conv4_flat = Flatten()(Layer_conv4)       
        Layer_Dense1 = Dropout(self.dropout)(Activation('relu')(BatchNormalization(axis=1)(Dense(1024, use_bias=False)(Layer_conv4_flat))))  
        Layer_Dense2 = Dropout(self.dropout)(Activation('relu')(BatchNormalization(axis=1)(Dense(512, use_bias=False)(Layer_Dense1))))          
        self.Network_policy = Dense(self.action_size, activation='softmax', name='Network_policy')(Layer_Dense2)   
        self.Network_value = Dense(1, activation='tanh', name='Network_value')(Layer_Dense2)                   
        self.model = Model(inputs=self.input_boards, outputs=[self.Network_policy, self.Network_value])
        self.model.compile(loss=['categorical_crossentropy','mean_squared_error'], optimizer=Adam(self.lr))
        
    #is called to train the model given an input training set in list format   
    def train(self, inputs, batch_size=50, epochs=50, cuda=False):
        inputs = np.asarray(inputs)
        input_boards=inputs[:,0:75]
        target_pis=inputs[:,75:150]
        target_vs=inputs[:,150:151]
        input_boards = np.asarray(input_boards)
        input_boards = input_boards.reshape(input_boards.shape[0],5,5, 3)
        target_pis = np.asarray(target_pis)
        target_vs = np.asarray(target_vs)
        self.model.fit(x = input_boards, y = [target_pis, target_vs], batch_size = batch_size, epochs = epochs) 
        return self
      
    #is called to predict a policy given an input Board   
    def predict_netoutput(self,Input_Board):
        Input_Board=Format_data(Input_Board)
        inputs = np.asarray(Input_Board)
        input_board = inputs.reshape(1,5,5, 3)
        pi,v=self.model.predict(input_board)
        return pi, v
    #is called to predict the best Network move given an Input Board
    def NNeT_Move(self,Input_Board):
        pi,v=self.predict_netoutput(Input_Board)
        pi=pi.flatten()
        pi=pi.tolist()
        possible = []
        liste_coups_possible = Input_Board.legalMoves ()
        for m in liste_coups_possible:
            i = m.code ()
            move_score = pi[i]
            possible.append(move_score)
        best_move = max(possible)
        return liste_coups_possible[possible.index(best_move)]
    

Using TensorFlow backend.


In [0]:
# This is needed to concatenate the training set (input,policy,value)
def Playout_state_results(State_list,State_Move,outcome): 
    playout_result=[]
    for i in range(len(State_Move)):
        result=State_list[i]
        x=State_Move[i]
        x.append(outcome)
        result+=x
        playout_result.append(result)
    return playout_result

#This plays one game episodes given a neural network 
def Execute_game_episode(NNeT):
    game=Board()
    condition=False
    State_list=[]
    State_Move=[]    
    State_list.append(Format_data(game)) #keep the starting state
    while condition==False:
        game.print()
        print('*'*10)
        CNN_Move = game.PUCT({},NNeT)
        print(CNN_Move)
        game.play (CNN_Move)
        if game.won(Black)==True:
            condition=True
            outcome=0
            game.play (CNN_Move)
            game.print()
            print('*'*10)
            break
        if game.won(White)==True:
            condition=True
            game.play (CNN_Move)
            game.print()
            print('*'*10)
            outcome=1
            break
        State_list.append(Format_data(game))
        State_Move.append(Format_data(game))
    State_Move.append(Format_data(game))
    Training_examples=Playout_state_results(State_list,State_Move,outcome)
    return Training_examples

In [0]:

#The below game gives a random move given an input Board
def Play_Random_Move(Input_Board):
    l = Input_Board.legalMoves ()
    indice = int (random.random () * len (l))
    m = l [indice]
    return m

#The below function is aimed at multiple playouts of 2 given players and it provides the possibility to have different agents given a type as follows
#Type_game=
#Battle_Nets (pit tow neural net players against each other)
#Net_Random (pit a neural net player against a random player), use as follow pit_NNets(Num_game_ep,Neuralnetwor,Type_game='Net_Random')
#Net_Montecarlo (pit a neural net player against a monte carlo player), use as follow pit_NNets(Num_game_ep,Neuralnetwor,Type_game='Net_Montecarlo')
#Montecarlo_Random (pit a montecarlo player against a random player), use as follow pit_NNets(Num_game_ep,Type_game='Net_Random')

def pit_NNets(Num_game_ep,WhiteNNeT=None,BlackNNeT=None,Type_game='Battle_Nets'):
    outcome_White=0
    outcome_Black=0 
    for i in range(Num_game_ep):
        condition=False
        game = Board ()
        while condition==False:
            if Type_game=='Battle_Nets':
                White_move=WhiteNNeT.NNeT_Move(game)
                Black_move=BlackNNeT.NNeT_Move(game)
            elif Type_game=='Net_Random':
                White_move=WhiteNNeT.NNeT_Move(game)
                Black_move=Play_Random_Move(game)
            elif Type_game=='Net_Montecarlo':
                White_move=WhiteNNeT.NNeT_Move(game)
                Black_move=game.flat(100)
            elif Type_game=='Montecarlo_Random':
                White_move=game.flat(100)
                Black_move=Play_Random_Move(game)
            if game.turn==White:
                game.play (White_move)
            else:
                game.play (Black_move)   
            if game.won(Black)==True:
                condition=True
                outcome_Black+=1
                break
            if game.won(White)==True:
                condition=True
                outcome_White+=1
                break    
    return (outcome_White/Num_game_ep)*100

In [0]:
#Below is the policy iteration to train the agent 
def policy_Iteration(Num_iter,Num_Epis,Win_threshold):
    # initialise random neural network
    NNeT=BreakthroughNNet() 
    tr_examples = []    
    for i in range(Num_iter):
        print('Iter:',i+1)
        for e in range(Num_Epis):
            print('Eps:',e+1)
            # collect training examples from this game
            tr_examples += Execute_game_episode(NNeT)    
        #train a new network
        New_NNeT=BreakthroughNNet() 
        New_NNeT.train(tr_examples)   
        #pit the old and new net against each other and select the best net who win a % of games higher than the given threshold 
        frac_win = pit_NNets(50,New_NNeT,NNeT)
        print('the new neuralnetwork player won',frac_win,'% of the played games', 'on Iteration Number:',i+1)
        if frac_win > Win_threshold: 
            NNeT = New_NNeT                                            
    return NNeT,tr_examples

In [0]:
# policy iteration with 5 iteration, 250 game episodes, and a win threshold of 52% of played games 
import time
start = time.time()
trained_network,training_set=policy_Iteration(5,250,52)
end = time.time()
print(end - start)

Black/3/3/2/4
|w|w|w| |w|
| |b| |w| |
|w|w| |w|b|
|b| | | | |
|b|b|b|b|b|
**********
White/0/1/1/2
|w| |w| |w|
| |b|w|w| |
|w|w| |w|b|
|b| | | | |
|b|b|b|b|b|
**********
Black/4/3/3/4
|w| |w| |w|
| |b|w|w| |
|w|w| |w|b|
|b| | | |b|
|b|b|b| |b|
**********
White/0/0/1/0
| | |w| |w|
|w|b|w|w| |
|w|w| |w|b|
|b| | | |b|
|b|b|b| |b|
**********
Black/4/2/3/3
| | |w| |w|
|w|b|w|w| |
|w|w| |w|b|
|b| | |b|b|
|b|b| | |b|
**********
White/1/3/2/2
| | |w| |w|
|w|b|w| | |
|w|w|w|w|b|
|b| | |b|b|
|b|b| | |b|
**********
Black/3/4/2/3
| | |w| |w|
|w|b|w| | |
|w|w|w|b|b|
|b| | |b| |
|b|b| | |b|
**********
White/2/2/3/1
| | |w| |w|
|w|b|w| | |
|w|w| |b|b|
|b|w| |b| |
|b|b| | |b|
**********
Black/4/1/3/2
| | |w| |w|
|w|b|w| | |
|w|w| |b|b|
|b|w|b|b| |
|b| | | |b|
**********
White/2/1/3/2
| | |w| |w|
|w|b|w| | |
|w| | |b|b|
|b|w|w|b| |
|b| | | |b|
**********
Black/2/4/1/4
| | |w| |w|
|w|b|w| |b|
|w| | |b| |
|b|w|w|b| |
|b| | | |b|
**********
White/3/1/4/2
| | |w| |w|
|w|b|w| |b|
|w| | |b| |
|b| |w|b| |
|b|

In [0]:
#saves the trained network
trained_network.model.save('Breakthrough_Trained_Nets') 
#saves the training set
np.savetxt('Breakthrough_Trained_Set.csv', training_set, delimiter=',')